<a href="https://colab.research.google.com/github/anscch/ATPAD-LAIDEA-UNAM/blob/main/METEO_ATPAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#***METEO_ATPAD***

#***DATASET CONSTRUCTION***

Parquet files contains all data from each station. This files will need to be updated as new information is available. Updated versions of such files can be builted using PREP_ATPAT. New data will be needed in METEO_RAW directory.

First cleaning steps are carried out by concatenating all station's files into a single sataset since all parameters are the same between stations.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import duckdb
connection = duckdb.connect()
import os
import datetime
import calendar

mor_data_raw = pd.read_parquet('/content/drive/MyDrive/ATPAD_COLAB/PARQUET_FILES/RUOA_MORE_1h_Meteo_2024.parquet').reset_index(drop=True)
sal_data_raw = pd.read_parquet('/content/drive/MyDrive/ATPAD_COLAB/PARQUET_FILES/RUOA_SLLO_1h_Meteo_2024.parquet').reset_index(drop=True)
agu_data_raw = pd.read_parquet('/content/drive/MyDrive/ATPAD_COLAB/PARQUET_FILES/RUOA_AGSC_1h_Meteo_2024.parquet').reset_index(drop=True)

#filling gaps in timestamp

dfs = [mor_data_raw, sal_data_raw, agu_data_raw]

for i in range(3):
    full_range = pd.date_range(start=dfs[i]['Time'].min(), end=dfs[i]['Time'].max(), freq='h')
    dfs[i]= dfs[i].set_index('Time')
    dfs[i]= dfs[i].reindex(full_range)

dfs[0]['region'] = 'Morelia'
dfs[1]['region'] = 'Saltillo'
dfs[2]['region'] = 'Aguascalientes'

# All regions together

meteo_raw = pd.concat(dfs).reset_index(drop=False)
meteo_raw = meteo_raw.rename(columns={'index': 'Time'})
meteo_raw

,Time,Temp_Avg,RH_Avg,WSpeed_Avg,WSpeed_Max,WDir_Avg,Rain_Tot,Press_Avg,Rad_Avg,region
0,2015-08-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Morelia
1,2015-08-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Morelia
2,2015-08-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Morelia
3,2015-08-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Morelia
4,2015-08-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Morelia
...,...,...,...,...,...,...,...,...,...,...
248494,2024-11-10 23:00:00,21.63,37.71,1.317,3.25,3.0,0.0,817.400,-1.165,Aguascalientes
248495,2024-11-11 00:00:00,20.20,42.13,1.414,3.34,8.6,0.0,817.702,-0.268,Aguascalientes
248496,2024-11-11 01:00:00,19.39,44.20,2.281,4.22,28.3,0.0,817.713,-0.553,Aguascalientes
248497,2024-11-11 02:00:00,19.43,43.02,1.840,3.55,39.6,0.0,817.628,-0.272,Aguascalientes


#***PARAMETERS FOR VALIDATION***

Statistics are calculated by region for further data validation and outlier detection.

In [ ]:
from scipy import stats

def q005(arr):
    return arr.quantile(0.005)

def q01(arr):
    return arr.quantile(0.01)

def q995(arr):
    return arr.quantile(0.995)

def q995(arr):
    return arr.quantile(0.995)

def q98(arr):
    return arr.quantile(0.98)

def q99(arr):
    return arr.quantile(0.99)


dft = meteo_raw.groupby('region').agg(WSpeed_AVG_qh=('WSpeed_Avg', q99),
                            WSpeed_Max_qh=('WSpeed_Max', q98),
                            press_avg=('Press_Avg', 'mean'),
                            press_std=('Press_Avg', 'std'),
                            wspeed_avg=('WSpeed_Avg', 'mean'),
                            wspeed_std=('WSpeed_Avg', 'std'),
                            temp_avg=('Temp_Avg', 'mean'),
                            temp_std=('Temp_Avg', 'std'),
                            ).reset_index()

dft

,region,WSpeed_AVG_qh,WSpeed_Max_qh,press_avg,press_std,wspeed_avg,wspeed_std,temp_avg,temp_std
0,Aguascalientes,6.18982,11.5700,816.784258,2.371658,2.431338,1.209265,19.263423,5.737975
1,Morelia,5.12500,10.8018,806.011108,1.955770,1.686199,1.232207,17.639876,5.049476
2,Saltillo,7.52750,13.4800,823.902007,2.970401,3.102591,7.037040,18.298960,6.178480


#***CLEANING DATASET***

z-scores are calculated using data dft dataframe for each datapoint in meteo_raw by region. Then the different validation criteria are evaluated.

In [ ]:
meteo_clean = connection.execute('''
with qz as (select *,
    (meteo_raw.Press_Avg - dft.press_avg)/dft.press_std as press_zvalue,
    (meteo_raw.Temp_Avg - dft.temp_avg)/dft.temp_std as temp_zvalue,
    (meteo_raw.WSpeed_Avg - dft.wspeed_avg)/dft.wspeed_std as wspeed_zvalue,

    from meteo_raw

    left join dft on dft.region = meteo_raw.region)

select
    Time,
    region,
    CASE when
        (
            (temp_zvalue between -3 and 3) or (Temp_Avg is NULL)
        )
        then Temp_Avg else NULL
    END as Temp_Avg,


    CASE when
        (
            (wspeed_zvalue between -4 and 4) or (WSpeed_AVG is NULL)
        )
        then WSpeed_AVG else NULL
    END as WSpeed_AVG,

    CASE when
        (
            (WSpeed_Max between 0 and WSpeed_Max_qh) or (WSpeed_Max is NULL)
        )
        then WSpeed_Max else NULL
    END as WSpeed_Max,

    CASE when
        (
            (Rain_Tot >= 0) or (Rain_Tot is NULL)
        )
        then Rain_Tot else NULL
    END as Rain_Tot,

    CASE when
        (
            (press_zvalue between -3 and 3) or (Press_Avg is NULL)
        )
        then Press_Avg else NULL
    END as Press_Avg,

    CASE when
        (
            (Rad_Avg > 0.001) or (Rad_Avg is NULL)
        )
        then Rad_Avg else NULL
    END as Rad_Avg,


    CASE when
        (
            (RH_Avg between 1 and 99) or (RH_Avg is NULL)
        )
        then RH_Avg else NULL
    END as RH_Avg,

    CASE when
        (
            (WDir_AVG between 0.001 and 359.999) or (WDir_AVG is NULL)
        )
        then WDir_AVG else NULL
    END as WDir_AVG


from qz
''').df()

meteo_clean

,Time,region,Temp_Avg,WSpeed_AVG,WSpeed_Max,Rain_Tot,Press_Avg,Rad_Avg,RH_Avg,WDir_AVG
0,2015-08-01 00:00:00,Morelia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-08-01 01:00:00,Morelia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-08-01 02:00:00,Morelia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-08-01 03:00:00,Morelia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-08-01 04:00:00,Morelia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
248494,2024-11-10 23:00:00,Aguascalientes,21.63,1.317,3.25,0.0,817.400,NaN,37.71,3.0
248495,2024-11-11 00:00:00,Aguascalientes,20.20,1.414,3.34,0.0,817.702,NaN,42.13,8.6
248496,2024-11-11 01:00:00,Aguascalientes,19.39,2.281,4.22,0.0,817.713,NaN,44.20,28.3
248497,2024-11-11 02:00:00,Aguascalientes,19.43,1.840,3.55,0.0,817.628,NaN,43.02,39.6




**SORTING DATA BY DATE**

Data is finally sorted by date. Pay attention to regions sice timestamp will repeat dates for each region. If you want to save this dataframe as csv file, then you should run the last section SAVING CLEANED DATASET AS CSV.


In [ ]:
meteo_clean = meteo_clean.sort_values(by = 'Time')

meteo_clean


,Time,region,Temp_Avg,WSpeed_AVG,WSpeed_Max,Rain_Tot,Press_Avg,Rad_Avg,RH_Avg,WDir_AVG
107583,2015-05-01 00:00:00,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
163276,2015-05-01 00:00:00,Saltillo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107584,2015-05-01 01:00:00,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
163277,2015-05-01 01:00:00,Saltillo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
163278,2015-05-01 02:00:00,Saltillo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
107581,2024-11-11 01:00:00,Saltillo,13.49,2.295,3.85,0.0,825.039,NaN,86.58,168.9
248496,2024-11-11 01:00:00,Aguascalientes,19.39,2.281,4.22,0.0,817.713,NaN,44.20,28.3
248497,2024-11-11 02:00:00,Aguascalientes,19.43,1.840,3.55,0.0,817.628,NaN,43.02,39.6
107582,2024-11-11 02:00:00,Saltillo,12.98,1.680,3.09,0.0,824.802,NaN,79.42,124.2


#***SPLIT DATA BY REGION***

Uncomment the line for the region you want.

In [ ]:
#meteo_clean = meteo_clean.loc[meteo_clean['region']=='Morelia']
#meteo_clean = meteo_clean.loc[meteo_clean['region']=='Saltillo']
meteo_clean = meteo_clean.loc[meteo_clean['region']=='Aguascalientes']



meteo_clean


,Time,region,Temp_Avg,WSpeed_AVG,WSpeed_Max,Rain_Tot,Press_Avg,Rad_Avg,RH_Avg,WDir_AVG
107583,2015-05-01 00:00:00,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107584,2015-05-01 01:00:00,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107585,2015-05-01 02:00:00,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107586,2015-05-01 03:00:00,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107587,2015-05-01 04:00:00,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
248494,2024-11-10 23:00:00,Aguascalientes,21.63,1.317,3.25,0.0,817.400,NaN,37.71,3.0
248495,2024-11-11 00:00:00,Aguascalientes,20.20,1.414,3.34,0.0,817.702,NaN,42.13,8.6
248496,2024-11-11 01:00:00,Aguascalientes,19.39,2.281,4.22,0.0,817.713,NaN,44.20,28.3
248497,2024-11-11 02:00:00,Aguascalientes,19.43,1.840,3.55,0.0,817.628,NaN,43.02,39.6


#***SELECT SPECIFIC TIME PERIOD***

Insert speficfic start and end dates including time information following the format YYY-mm-dd HH:MM:SS

In [ ]:
start_date ='2016-01-01 00:00:00'
end_date = '2016-12-31 23:00:00'

period = (meteo_clean['Time'] >= start_date) & (meteo_clean['Time'] <= end_date)

meteo_clean = meteo_clean.loc[period]

meteo_clean

,Time,region,Temp_Avg,WSpeed_AVG,WSpeed_Max,Rain_Tot,Press_Avg,Rad_Avg,RH_Avg,WDir_AVG
138039,2016-01-01 00:00:00,Aguascalientes,15.57,1.612,NaN,0.0,815.0,NaN,64.90,17.1
138040,2016-01-01 01:00:00,Aguascalientes,13.95,0.845,NaN,0.0,814.0,NaN,71.67,354.3
138041,2016-01-01 02:00:00,Aguascalientes,13.06,1.713,NaN,0.0,814.0,NaN,74.56,14.2
138042,2016-01-01 03:00:00,Aguascalientes,12.13,1.781,NaN,0.0,814.0,NaN,76.34,15.5
138043,2016-01-01 04:00:00,Aguascalientes,11.24,1.957,NaN,0.0,814.0,NaN,80.22,22.9
...,...,...,...,...,...,...,...,...,...,...
157058,2016-12-31 19:00:00,Aguascalientes,19.73,2.460,NaN,0.0,815.0,3.7,39.15,279.8
157059,2016-12-31 20:00:00,Aguascalientes,18.37,2.147,NaN,0.0,815.2,NaN,43.65,267.0
157060,2016-12-31 21:00:00,Aguascalientes,17.75,2.023,NaN,0.0,816.0,NaN,45.81,276.8
157061,2016-12-31 22:00:00,Aguascalientes,16.96,1.511,NaN,0.0,816.2,NaN,47.52,277.7


#***24h AVERAGE AND DIURNAL DATA***

#24-hour Average

The calculation considers a completeness criterion to preserve data representativeness. The 24-hour mean is calculated only if there is a minimum of 18 hours of data available for each day. Additionally, a special method is applied to calculate the mean wind direction, accounting for its circular nature. The results are stored in a new DataFrame called daily_mean, which can be saved as a CSV file using the code in the next cell.

#Diurnal Data
For better data representation, the diurnal values are computed as the median of each parameter.


In [20]:
'24h average'

meteo_clean=meteo_clean.assign(Date=meteo_clean.Time.dt.date)   # Create column with dates only

def calculate_daily_mean(group, is_circular=False):   # is_circular conditional for circular variables such as wind direction. To be declared on function custom_mean.
  valid_count = group.notna().sum()
  if valid_count >= 18:
    if is_circular:
      rad = np.deg2rad(group)
      mean_sin = np.mean(np.sin(rad))
      mean_cos = np.mean(np.cos(rad))
      mean_angle = np.arctan2(mean_sin, mean_cos)
      return np.rad2deg(mean_angle).round(2) % 360     # Ensure that mean_angle is between 0 an 360
    else:
      return group.mean().round(2)
  else:
    return np.nan

def custom_mean(group):
  return {
      'Temp_Avg': calculate_daily_mean(group['Temp_Avg']),
      'WSpeed_AVG': calculate_daily_mean(group['WSpeed_AVG']),
      'WSpeed_Max': calculate_daily_mean(group['WSpeed_Max']),
      'Rain_Tot': calculate_daily_mean(group['Rain_Tot']),
      'Press_Avg': calculate_daily_mean(group['Press_Avg']),
      'Rad_Avg': calculate_daily_mean(group['Rad_Avg']),
      'RH_Avg': calculate_daily_mean(group['RH_Avg']),
      'WDir_AVG': calculate_daily_mean(group['WDir_AVG'], is_circular=True),
  }

daily_mean = meteo_clean.groupby(['region', 'Date']).apply(custom_mean, include_groups=False).apply(pd.Series)
daily_mean = daily_mean.reset_index()
display(daily_mean)

del meteo_clean['Date']

'hourly average (output: 24 lines) diurnal cycles taking the median for better representation'

# meteo_clean = meteo_clean.groupby(['region', meteo_clean['Time'].dt.hour]).median().reset_index(drop=True)
# meteo_clean

,region,Date,Temp_Avg,WSpeed_AVG,WSpeed_Max,Rain_Tot,Press_Avg,Rad_Avg,RH_Avg,WDir_AVG
0,Aguascalientes,2016-01-01,15.98,2.42,NaN,0.0,814.00,NaN,67.26,16.72
1,Aguascalientes,2016-01-02,15.84,1.93,NaN,0.0,814.00,NaN,68.59,317.77
2,Aguascalientes,2016-01-03,15.16,2.65,NaN,0.0,815.25,NaN,56.53,4.14
3,Aguascalientes,2016-01-04,14.07,2.76,NaN,0.0,817.12,NaN,46.32,340.82
4,Aguascalientes,2016-01-05,13.00,2.54,NaN,0.0,818.58,NaN,44.22,285.98
...,...,...,...,...,...,...,...,...,...,...
361,Aguascalientes,2016-12-27,16.58,2.40,NaN,0.0,819.75,NaN,52.16,23.65
362,Aguascalientes,2016-12-28,15.79,2.26,NaN,0.0,819.68,NaN,55.62,26.75
363,Aguascalientes,2016-12-29,16.16,3.23,NaN,0.0,819.86,NaN,51.49,48.73
364,Aguascalientes,2016-12-30,15.89,2.15,NaN,0.0,819.66,NaN,56.31,74.41


'hourly average (output: 24 lines) diurnal cycles taking the median for better representation'

#***SAVING CLEANED DATASET AS CSV FILE***

In [21]:
date_now = datetime.datetime.now() - datetime.timedelta(hours=6)
date_now = date_now.strftime('%Y-%m-%d %H:%M')

file_name = 'meteo_clean_' + str(date_now) + '.csv'

meteo_clean.to_csv('/content/drive/MyDrive/ATPAD_COLAB/OUTPUT_FILES/'+ file_name)

#daily_mean.to_csv('/content/drive/MyDrive/ATPAD_COLAB/OUTPUT_FILES/daily_mean_meteo_' + str(date_now) + '.csv')